07 May 2020

# An example: running RankCorr on Paul

For editing packages - don't need to run this

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

Also load scanpy for easy access to the Paul data set.  Check out the scanpy repository at https://github.com/theislab/scanpy

In [5]:
import scanpy.api as sc

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')  # low dpi (dots per inch) yields small inline figures
sc.logging.print_versions()

/home/ahsvargo/miniconda3/envs/bmc/lib/python3.8/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex
/home/ahsvargo/miniconda3/envs/bmc/lib/python3.8/site-packages/scanpy/api/__init__.py:2: FutureWarning: 

In a future version of Scanpy, `scanpy.api` will be removed.
Simply use `import scanpy as sc` and `import scanpy.external as sce` instead.

  warnings.warn(


scanpy==1.4.6 anndata==0.7.1 umap==0.4.0 numpy==1.18.1 scipy==1.4.1 pandas==1.0.3 scikit-learn==0.22.1 statsmodels==0.11.0 python-igraph==0.8.0


In [6]:
import anndata

## Load the RankCorr methods

The RankCorr code is currently in a heavily modified version of the PicturedRocks package.  See the PicturedRocks repo at https://github.com/umangv/picturedrocks for the original package.

The modified package is included in the code here - this needs to be loading the local version for the remainder of the code to run

In [4]:
from picturedRocks import Rocks

Required inputs for the `Rocks` class:

* `X`, an `np.ndarry` of gene counts.  Each row should contain the genetic information from a cell; the columns of `X` correspond to the genes (note that this is the transpose of some commonly used packages).
* `y`, a vector of cluster labels.  These labels must be consecutive integers starting at 0.


## Load the Paul dataset

This will automatically download the data set if this is your first time running it.

In [12]:
dataset = "paul15"

In [7]:
adata = sc.datasets.paul15()

... storing 'paul15_clusters' as categorical
Trying to set attribute `.uns` of view, copying.


In [9]:
adata

AnnData object with n_obs × n_vars = 2730 × 3451 
    obs: 'paul15_clusters'
    uns: 'iroot'

Create the required vector of cluster labels based on the strings provided in the AnnData object.

In [10]:
lookup = list(adata.obs['paul15_clusters'].cat.categories)
yVec = np.array([lookup.index( adata.obs['paul15_clusters'][i] ) for i in range(adata.obs['paul15_clusters'].shape[0]) ])

Here are cluster names from the Paul data set.  See Paul (2015).

In [14]:
lookup

['1Ery',
 '2Ery',
 '3Ery',
 '4Ery',
 '5Ery',
 '6Ery',
 '7MEP',
 '8Mk',
 '9GMP',
 '10GMP',
 '11DC',
 '12Baso',
 '13Baso',
 '14Mo',
 '15Mo',
 '16Neu',
 '17Neu',
 '18Eos',
 '19Lymph']

Create the `Rocks` object as outlined above

In [14]:
data = Rocks(adata.X, yVec)

# PicturedRocks provides normalization capabilities, though this shouldn't be used for marker selection. 
#data.normalize(log=False, totalexpr=10000)

'''
# It is also possible to use the PicturedRocks for fold testing, to match the results from the manuscript. 
# This will be discussed more in the future.
ft = FoldTester(data)
folds = np.load("paul15-scviFolds.npz")["folds"]
ft.folds = folds
ft.validatefolds()

ft.makerocks(verbose=0)
'''

'\n# It is also possible to use the PicturedRocks for fold testing, to match the results from the manuscript. \n# This will be discussed more in the future.\nft = FoldTester(data)\nfolds = np.load("paul15-scviFolds.npz")["folds"]\nft.folds = folds\nft.validatefolds()\n\nft.makerocks(verbose=0)\n'

## Run RankCorr

The main RankCorr method is `CSrankMarkers.`  In addition to the data provided by the `Rocks` object, it requires one parameter:

* `lamb` is the sparsity parameter - larger values of `lamb` will result in more markers selected per cluster

There are several optional boolean parameters:

* `writeOut` defaults to `False` and controls whether or not to write the selected markers to a file.  The deafult filename is "ovrRankGenes-lamb{}.dat", with the input value of `lamb`.
* `keepZeros` should almost always be set to `False` (the default value).  It provides a tweak to keep the in the data matrix `X` unchanged by the ranking procedure (i.e. the zeros will be mapped to zero).  This has the effect of removing the zero counts from the analysis (while ranking all of the other counts correctly) and is purely added for experimental exploration.
* `onlyNonZero` should almost always be set to `False` (the default value).  This provides a tweak to only rank the nonzero counts, pretending that the zero counts did not even exist. This is only useful if the zero counts in the application are completely uninformative (e.g. a zero count could easily represent a complete erasure of a massive count) which is not the case in UMI counts scRNA-seq data.

Note that there are really not any hyperparamters to tweak!

In [22]:
lamb = 3.0 # this can be whatever

%time markers = data.CSrankMarkers(lamb=lamb, writeOut=False, keepZeros=False, onlyNonZero=False)

CPU times: user 12.7 ms, sys: 782 µs, total: 13.5 ms
Wall time: 12.9 ms


By deafault, this gives a list of markers for the whole clustering, without separating markers by the cluster that they are selected for.  If `writeOut = True`, the cluster information is stored in the output file.

In [21]:
len(markers)

84

If you have the geneNames, add them to the `Rocks` object - then these markers can be converted to gene names.

In [24]:
geneNames = np.array(adata.var.index)
data.genes = geneNames

In [25]:
marker_genes = data.markers_to_genes(markers)

In [26]:
marker_genes[:10]

['1100001G20Rik',
 'Elane',
 'Calr',
 'Emb',
 'Rhd',
 'Car1',
 'Car2',
 'Epx',
 'Ermap',
 'Casp3']